In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp27-none-any.whl size=5888 sha256=a9ea3848c9ff27a786372fe88c1e7c462701e9fcbe0851545bbd82ad683e7c41
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import tensorflow as tf
import librosa
import pickle
import numpy as np
from python_speech_features import mfcc

In [0]:
#importing traning mfccs and labels
with open('/content/gdrive/My Drive/Colab Notebooks/mfcc_train.pkl', 'rb') as m:
    mfcc_train = pickle.load(m)
with open('/content/gdrive/My Drive/Colab Notebooks/train_labels.pkl', 'rb') as n:
    train_labels = pickle.load(n)

#importing valid mfccs and labels
with open('/content/gdrive/My Drive/Colab Notebooks/mfcc_valid.pkl', 'rb') as o:
    mfcc_valid = pickle.load(o)
with open('/content/gdrive/My Drive/Colab Notebooks/valid_labels.pkl', 'rb') as p:
    valid_labels = pickle.load(p)
    
#importing test mfccs and labels
with open('/content/gdrive/My Drive/Colab Notebooks/mfcc_test.pkl', 'rb') as q:
    mfcc_test = pickle.load(q)
with open('/content/gdrive/My Drive/Colab Notebooks/test_labels.pkl', 'rb') as r:
    test_labels = pickle.load(r)

In [0]:
#Labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(train_labels)
y_valid = le.fit_transform(valid_labels)
y_test = le.fit_transform(test_labels)
classes = list(le.classes_)
len(classes)

7

In [0]:
#converting labels to one hot encoding
from keras.utils import np_utils
y_train=np_utils.to_categorical(y_train, num_classes=len(classes))
y_valid=np_utils.to_categorical(y_valid, num_classes=len(classes))
y_test=np_utils.to_categorical(y_test, num_classes=len(classes))

Using TensorFlow backend.


In [0]:
#Check the shape of mfcc array
mfcc_train = np.array(mfcc_train)
mfcc_valid = np.array(mfcc_valid)
mfcc_test = np.array(mfcc_test)

In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# Feature Scaling on training dataset
num_instances, num_time_steps, num_features = mfcc_train.shape
mfcc_train = np.reshape(mfcc_train, newshape=(-1, num_features))
mfcc_train = sc.fit_transform(mfcc_train)
mfcc_train = np.reshape(mfcc_train, newshape=(num_instances, num_time_steps, num_features))
# Feature Scaling on validation dataset
num_instances, num_time_steps, num_features = mfcc_valid.shape
mfcc_valid = np.reshape(mfcc_valid, newshape=(-1, num_features))
mfcc_valid = sc.transform(mfcc_valid)
mfcc_valid = np.reshape(mfcc_valid, newshape=(num_instances, num_time_steps, num_features))
# Feature Scaling on testing dataset
num_instances, num_time_steps, num_features = mfcc_test.shape
mfcc_test = np.reshape(mfcc_test, newshape=(-1, num_features))
mfcc_test = sc.fit_transform(mfcc_test)
mfcc_test = np.reshape(mfcc_test, newshape=(num_instances, num_time_steps, num_features))

In [0]:
#RNN Model
from tensorflow.contrib import rnn

In [0]:
n_input = mfcc_train.shape[2] #MFCC features
n_steps = mfcc_train.shape[1] #no. of timesteps
n_hidden = 64
n_classes = len(classes)
batch_size = 100
num_epochs = 100
no_of_batches = 100
#num_of_layers = 2

In [0]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
weights = {
   'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
   'out': tf.Variable(tf.random_normal([n_classes]))
}
keep_prob = tf.placeholder(tf.float32)

In [0]:
#batch
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [0]:
#RNN declaration
def RNN(x, weights, biases):
    x = tf.unstack(x, n_steps, 1)

   # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias = 0.2)
    
   #Dropout
   #lstm_dropout = rnn.DropoutWrapper(lstm_cell, output_keep_prob = 0.2)
   
   # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype = tf.float32)

   # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [0]:
#prediction
pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
saver = tf.train.Saver()

W0906 06:41:20.257649 140360891025280 deprecation.py:323] From <ipython-input-15-959b7414dfd7>:5: __init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0906 06:41:20.259623 140360891025280 deprecation.py:323] From <ipython-input-15-959b7414dfd7>:11: static_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
W0906 06:41:20.288933 140360891025280 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing 

In [0]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        loss_train = 0;
        loss_valid = 0;
        for i in range(no_of_batches):  
            batch_x, batch_y = next_batch(batch_size, mfcc_train, y_train)
            batch_x_val, batch_y_val = next_batch(batch_size, mfcc_valid, y_valid)
            batch_x = batch_x.reshape((batch_size, n_steps, n_input))
            batch_x_val = batch_x_val.reshape((batch_size, n_steps, n_input))
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
            training_acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            valid_acc = sess.run(accuracy, feed_dict={x: batch_x_val, y: batch_y_val})
            loss_train += sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            loss_valid += sess.run(cost, feed_dict={x: batch_x_val, y: batch_y_val})
        avg_loss_training = loss_train / no_of_batches
        avg_loss_validation = loss_valid / no_of_batches
        print("Epoch " + str(epoch) + ", Training Loss= " + \
              "{:.6f}".format(avg_loss_training) + ", Training Accuracy= " + \
              "{:.5f}".format(training_acc) + ", Validation Loss= " + \
              "{:.5f}".format(avg_loss_validation) + ", Validation Accuracy= " + \
              "{:.5f}".format(valid_acc))
    saver.save(sess, 'content/my_test_model')   
    print("Optimization Finished!")
    validate_feed = {x: mfcc_valid, y: y_valid}
    print("Validation Accuracy:", \
        sess.run(accuracy, feed_dict=validate_feed))
    test_feed = {x:mfcc_test , y: y_test}
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict=test_feed))

Epoch 0, Training Loss= 1.125884, Training Accuracy= 0.83000, Validation Loss= 1.12737, Validation Accuracy= 0.83000
Epoch 1, Training Loss= 0.815815, Training Accuracy= 0.80000, Validation Loss= 0.77235, Validation Accuracy= 0.77000
Epoch 2, Training Loss= 0.764174, Training Accuracy= 0.82000, Validation Loss= 0.77281, Validation Accuracy= 0.77000
Epoch 3, Training Loss= 0.775539, Training Accuracy= 0.84000, Validation Loss= 0.77249, Validation Accuracy= 0.78000
Epoch 4, Training Loss= 0.773679, Training Accuracy= 0.78000, Validation Loss= 0.72272, Validation Accuracy= 0.84000
Epoch 5, Training Loss= 0.736422, Training Accuracy= 0.82000, Validation Loss= 0.73972, Validation Accuracy= 0.87000
Epoch 6, Training Loss= 0.728480, Training Accuracy= 0.83000, Validation Loss= 0.71244, Validation Accuracy= 0.82000
Epoch 7, Training Loss= 0.694576, Training Accuracy= 0.78000, Validation Loss= 0.70887, Validation Accuracy= 0.80000
Epoch 8, Training Loss= 0.653530, Training Accuracy= 0.85000, Va

In [0]:
#Single prediction
#importing file
train_audio_path = '/content/gdrive/My Drive/Colab Notebooks/'
samples, sample_rate = librosa.load(train_audio_path+'fffcabd1_nohash_0.wav', sr = 16000)
#geting mfccs
mfcc1 = mfcc(samples, samplerate=sample_rate, numcep=13, nfilt=26, nfft = 512)
mfcc1 = mfcc1[np.newaxis, :, :]
#normalization of data
num_instances1, num_time_steps1, num_features1 = mfcc1.shape
mfcc1 = np.reshape(mfcc1, newshape=(-1, num_features1))
mfcc1 = sc.fit_transform(mfcc1)
mfcc1 = np.reshape(mfcc1, newshape=(num_instances1, num_time_steps1, num_features1))

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#Single prediction
#importing file
train_audio_path = '/content/gdrive/My Drive/Colab Notebooks/'
samples, sample_rate = librosa.load(train_audio_path+'0c40e715_nohash_0.wav', sr = 16000)
#geting mfccs
mfcc2 = mfcc(samples, samplerate=sample_rate, numcep=13, nfilt=26, nfft = 512)
mfcc2 = mfcc2[np.newaxis, :, :]
#normalization of data
num_instances2, num_time_steps2, num_features2 = mfcc2.shape
mfcc2 = np.reshape(mfcc2, newshape=(-1, num_features2))
mfcc2 = sc.fit_transform(mfcc2)
mfcc2 = np.reshape(mfcc2, newshape=(num_instances2, num_time_steps2, num_features2))

In [0]:
#my_voice example 'bed'
train_audio_path = '/content/gdrive/My Drive/Colab Notebooks/my_voice/'
samples, sample_rate = librosa.load(train_audio_path+'dog3.wav', sr = 16000)
#geting mfccs
mfcc_bed = mfcc(samples, samplerate=sample_rate, numcep=13, nfilt=26, nfft = 512)
mfcc_bed = mfcc_bed[np.newaxis, :, :]
#normalization of data
num_instances_bed, num_time_steps_bed, num_features_bed = mfcc_bed.shape
mfcc_bed = np.reshape(mfcc_bed, newshape=(-1, num_features_bed))
mfcc_bed = sc.fit_transform(mfcc_bed)
mfcc_bed = np.reshape(mfcc_bed, newshape=(num_instances_bed, num_time_steps_bed, num_features_bed))

In [0]:
with tf.Session() as sess:
  # Restore variables from disk.
  sess.run(init)
  saver.restore(sess, "my_test_model")
  print("Model restored.")
  y_example_bed = tf.nn.softmax(pred)
  y_pred_bed = sess.run(y_example_bed, feed_dict={x: mfcc_bed})
  print(y_pred_bed)

Model restored.
[[4.344317e-06 2.868859e-08 9.253901e-01 5.992488e-06 5.713409e-04
  7.402736e-02 7.359995e-07]]


In [0]:
result_bed = np.where(y_pred_bed == y_pred_bed.max())
print(result_bed)

(array([0]), array([2]))


In [0]:
classes

['bed', 'cat', 'dog', 'one', 'stop', 'unknown', 'zero']

In [0]:
#my_voice example 'john'
train_audio_path = '/content/gdrive/My Drive/Colab Notebooks/my_voice/'
samples, sample_rate = librosa.load(train_audio_path+'stop1.wav', sr = 16000)
#geting mfccs
mfcc_john = mfcc(samples, samplerate=sample_rate, numcep=13, nfilt=26, nfft = 512)
mfcc_john = mfcc_john[np.newaxis, :, :]
#normalization of data
num_instances_john, num_time_steps_john, num_features_john = mfcc_john.shape
mfcc_john = np.reshape(mfcc_john, newshape=(-1, num_features_john))
mfcc_john = sc.fit_transform(mfcc_john)
mfcc_john = np.reshape(mfcc_john, newshape=(num_instances_john, num_time_steps_john, num_features_john))

In [0]:
with tf.Session() as sess:
  # Restore variables from disk.
  sess.run(init)
  saver.restore(sess, "my_test_model")
  print("Model restored.")
  y_example_john = tf.nn.softmax(pred)
  y_pred_john = sess.run(y_example_john, feed_dict={x: mfcc_john})
  print(y_pred_john)

Model restored.
[[9.0181529e-10 2.1667731e-06 8.9769623e-05 1.0510327e-06 9.9987519e-01
  3.0381114e-05 1.4273786e-06]]


In [0]:
result_john = np.where(y_pred_john == y_pred_john.max())
print(result_john)

(array([0]), array([4]))


In [0]:
classes

['bed', 'cat', 'dog', 'one', 'stop', 'unknown', 'zero']

In [0]:
with tf.Session() as sess:
  # Restore variables from disk.
  sess.run(init)
  saver.restore(sess, "my_test_model")
  print("Model restored.")
  y_example = tf.nn.softmax(pred)
  y_pred = sess.run(y_example, feed_dict={x: mfcc1})
  print(y_pred)

Model restored.
[[1.2210573e-06 5.5986402e-07 1.1723056e-06 1.7017009e-06 6.7940624e-07
  9.2707167e-04 9.9906760e-01]]


In [0]:
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "content/my_test_model")
  print("Model restored.")
  y_example2 = tf.nn.softmax(pred)
  y_pred2 = sess.run(y_example2, feed_dict={x: mfcc2})
  print(y_pred2)

In [0]:
result2 = np.where(y_pred2 == y_pred2.max())
print(result2)

(array([0]), array([0]))


In [0]:
result = np.where(y_pred == y_pred.max())
print(result)

(array([0]), array([6]))


In [0]:
classes

['bed', 'cat', 'dog', 'one', 'stop', 'unknown', 'zero']